# Create Cluster: RDB
This notebook will create (start) an RDB cluster.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *

# Managed KX Database and Cluster names to create
DB_NAME="welcomedb"

SEC_THREADS='8'
CLUSTER_NAME=f"RDB_{DB_NAME}"

# Cluster Settings
CODEBASE="code"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.2xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':200 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }

INIT_SCRIPT=f'{CODEBASE}/init.q'
CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE} 
]

In [3]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS sessio: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Create Cluster

In [4]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_cluster_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    commandLineArguments=CMD_ARGS,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [5]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: RDB_welcomedb status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:05:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:05:30, waiting 30 sec ...
Cluster: RDB_welc

In [6]:
# Give permissions time to propogate after cluster creation....
time.sleep(20)

conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

:tcps://vpce-05b63a51a83e60849-uv0ahjts.vpce-svc-04530e89e89106ae7.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-05b63a51a83e60849-uv0ahjts.vpce-svc-04530e89e89106ae7.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGAaCXVzLWVhc3QtMSJHMEUCIQDmkYtsmRfdqJhnt%2B1FiI5WKdHONtoMwHw4hderK%2B%2FLrQIgQZ0J6JOcHETph0UVy2uPrCci69Kbvg81MijpEy5zXmsqgQMImP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw2MTI4NDEzODM1OTQiDHt5656U9uKyT1NZFyrVAlGZ06yszxXB59LXmB8tQTzk42U%2FFw7dwqUxhZNIPwxFDu5x0dDFRJxffolfudoe3E3r38tweK0nKf0vOhw9zgTzr7K887%2Fgm2kbHQA0ZuCxcVaadqU1Nbwun2SwldT%2Fl17NExt56yFD2WUEKMeYmxZamWOlcuTz%2Fpx6V0OCDtNjlJyGhwGhu5mzGSn2X3lqUSCTK%2B8p2pOSfe8q5X0HNXC3d%2B6uWjyL2bhLnKdyG1%2BDzI2flxONx2wleQ6zm72wZn9BpaPBJGKQsoKy1sAl8UXihkEJZg2dATIBUnW%2BkzUJB%2FxHL5mbIQ2dMGZUmbsLJtIZMIIH51QPcz4y%2BF7hNrKeyPxbAYhXcjzIiwkv990Uslet%2FaBweB2lZmYYB7bvFSsCkQJA736Z20c4hsILcs9%2BtdKSPHEoFC6wbiwKGWFy%2B9Uim%2FR1a20%2BQJEgW7U4%2B%2FHgHDu0MP%2BE2qMGOr8B9cHY7a

In [7]:
cdf = get_clusters(client, ENV_ID)

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
HDB_TAQ_2021_2D,CREATING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'TAQ_2021_2D'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_2D notebook,2023-05-30 23:09:47.415000+00:00,2023-05-30 23:09:30.508000+00:00,TAQ_2021_2D,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/']}]"
HDB_welcomedb,CREATING,HDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_HDB notebook,2023-05-30 22:56:22.186000+00:00,2023-05-30 22:56:11.773000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
RDB_welcomedb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_RDB notebook,2023-05-30 23:10:28.477000+00:00,2023-05-30 22:56:52.812000+00:00,welcomedb,[]
cluster_create_delete_db,DELETING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'create_delete_db'}, {'key': 'codebase', 'value': 'welcomedb'}]",Demo Cluster for database create_delete_db,2023-05-30 22:48:35.185000+00:00,2023-05-30 22:17:04.188000+00:00,create_delete_db,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
cluster_welcomedb,CREATING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'welcomedb'}]",Demo Cluster for database welcomedb,2023-05-30 22:53:48.399000+00:00,2023-05-30 22:53:31.356000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"


In [8]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-05-30 23:10:58.142926
